<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/%EA%B8%B0%EC%B4%88%ED%86%B5%EA%B3%84/naive_sol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 확률을 사용한 프로그래밍 프로젝트: 나이브 베이즈 스팸 메일 필터

## 나이브베이즈Naive Bayes

- 주어진 문서 $X$를 스팸인지 햄(정상 메일)인지 구분하기 위해 베이즈 정리를 이용하는 모델 나이브베이즈를 구현해본다.


$$
P(C \mid X) = \frac{P(X \mid C)P(C)}{P(X)}
$$

- 위 베이즈 정리에서 $P(C)$는 클래스의 사전확률prior로 스팸 메일 필터에서 클래스 변수 $C$는 스팸($C=SPAM$) 또는 햄($C=HAM$)이 되며 문서를 스팸과 햄으로 나눌때 두 클래스간 비율이 된다.

- $P(X \mid C)$는 클래스 조건부 확률로 주어진 클래스에서 $X$가 얼마나 존재 할만한 지를 나타내는 값이다. 다르게 말하면 클래스 $C$에 대한 가능도likelihood 이다.

- $P(X)$는 문서 $X$가 존재할 확률이며 클래스 조건부 확률 $P(X \mid C)$를 $P(C)$에 대해 평균을 낸 값이다. 확률의 곱법칙과 합법칙을 사용하면 다음과 같다.

$$
P(X) = P(X \mid C=SPAM)P(C=SPAM) + P(X \mid C=HAM)P(C=HAM)
$$

- 마지막으로 $P(C \mid X)$는 사후확률posterior로써 결과적으로 알고 싶은 문서 $X$에 대한 원인의 확률이다.(이 문서가 어떤 클래스(상자)에서 발생했는가!)

- $P(C \mid X)$를 알기위해 $P(X \mid C)$와 $P(C)$를 알아야 한다. 사전확률과 가능도를 알면 위 베이즈 정리에 의해 사후 확률을 계산할 수 있다.

- 나이브베이즈는 $P(X \mid C)$의 계산을 위해 조건부 독립을 가정한다. 문서 $X$가 다음처럼 특징feature으로 변환되었을 때

$$
X := (x_1, x_2, ..., x_n)
$$

- 조건부 독립에 의해 다음처럼 가정하게 된다.

$$
P(X \mid C) = P(x_1, x_2, ..., x_n \mid C) = P(x_1 \mid C) \times P(x_1 \mid C) \times \cdots \times P(x_n \mid C)
$$

- 결론적으로 나이브베이즈를 학습한다는 것은 학습 데이터를 특징화 하고 그 특징을 독립으로 가정한 다음 클래스에 대한 가능도 $P(X \mid C)$를 계산하는 것이다.

In [ ]:
import random
import pandas as pd
import numpy as np
import re

- 제공된 데이터 파일을 판다스pandas 데이터프레임으로 로딩한다.

In [ ]:
# 데이터 파일 가져오기
!gdown 16FxXGZDQK_8C5nW9o5H5lUAVRBG2YwKa

Downloading...
From: https://drive.google.com/uc?id=16FxXGZDQK_8C5nW9o5H5lUAVRBG2YwKa
To: /content/spam.csv
100% 154k/154k [00:00<00:00, 80.4MB/s]


In [ ]:
data = pd.read_csv('spam.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3423 entries, 0 to 3422
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   subject  3423 non-null   object
 1   spam     3423 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 30.2+ KB


## 문서 특징화

- 다루는 데이터는 텍스트이므로 어떤 식으로든 이를 숫자로 바꿔야 한다.
- 여기서는 텍스트를 숫자로 바꾸는 가장 고전적이면서도 간단한 방법인 BoW(Bag of Words) 방식을 사용한다.
- BoW 구축 방법은 아래 순서를 따른다.
    - 텍스트를 토큰이라는 단위로 쪼갤 방법을 마련한다. 토큰이란 단어가 될 수 도 있고 단어가 더 쪼개진 더 작은 단위가 될 수 도 있다. 여기서는 가장 간단하게 공백을 기준으로 단어를 자르고 잘린 단어를 토큰으로 사용한다.
    - 모델이 사용할 전체 단어를 포함하고 있는 단어장vocabulary `V`를 구축한다. 이는 모든 학습 데이터에 있는 텍스트를 이전 단계에서 준비한 토큰화 방법으로 자른 다음 얻게 되는 모든 단어에서 중복을 제거하면 얻을 수 있다.
    - 숫자로 바꾸고자하는 텍스트를 토큰화하고 `V`에서 해당 토큰들이 있는지 확인한다.
    - 길이가 `V`의 개수와 같은 0으로 채워진 벡터를 마련하고 앞서 `V`에 존재하는 토큰의 위치에 1을 기록한다.
- 위와 같은 방법으로 텍스트를 특징화하면 텍스트는 길이에 상관없이 무조건 길이가 `V`의 개수와 같은 벡터로 바뀌게 된다.
- 또한 변환된 벡터는 대부분 0이고 듬성듬성 1이 채워진 희소한 벡터가 되며 원래 텍스트의 단어순서는 모두 무시된 상태가 된다.
- 이런 특징은 머신러닝 측면에서 썩 좋은 특징이라 할 수 없기 때문에 다른 더 우수한 텍스트-벡터 변환 방법들이 많이 존재한다.
- 아래는 방금 소개한 방법의 간단한 예를 보여준다.

    ```
    # 단어장
    V = ["is", "it", "it's", "over", "find", "base", "till", "untill"]

    # 바꾸고자 하는 문장
    text = "It ain't over till it's over."

    # 중복제거 토큰화
    tokens = ["it", "ain't", "over", "till", "it's"]

    # 특징화 벡터
    feature = [0, 1, 1, 1, 0, 0, 1, 0]
    ```


### tokenize


- 아래 `tokenize` 함수는 주어진 문장을 소문자로 만들고 공백 기준으로 토큰화 한 후 중복을 제거해 단어의 리스트를 반환한다.

In [ ]:
def tokenize(message):
    message = message.lower()                       # 모두 소문자로 바꿈
    all_words = re.findall("[a-z0-9']+", message)   # 숫자,알파벳(소문자)로 된 낱말만 추출
    return list(set(all_words))                     # 중복제거하고 되돌림

In [ ]:
# tokenizez 테스트
tokenize("This gives 101 true positives (spam classified as 'spam') @?")

['gives',
 'true',
 '101',
 "'spam'",
 'spam',
 'this',
 'positives',
 'as',
 'classified']

### 단어장 `V`

- 단어장 `V`를 구축하기 위해 데이터 셋을 학습과 테스트로 나눈다.
- 전체 데이터 개수가 3423개이므로 3000개, 423개 씩 나눈다.

In [ ]:
data_train = data[:3000]
data_test = data[3000:]

- 학습 데이터는 다음처럼 이메일의 제목과 타겟으로 구성되어있다.

- 타겟값은 Spam메일일 경우 True, 정상 메일이면 False이다.

In [ ]:
data_train

,subject,spam
0,[ILUG] Garden Ornaments | mvcmv,True
1,Re: mplayer not working for me,False
2,Re: exmh bug?,False
3,MacOS X (Re: [SAtalk] [OT] Habeas-talk (was: 2...,False
4,Re: girls,True
...,...,...
2995,[Lockergnome Digital Media] Humblest Alarm,False
2996,DVD capture: Unbreaking the Mac,False
2997,New light on a lost world of shattered icons,False
2998,Re: [meta-forkage],False


- `data_train`에 있는 모든 샘플들에 대해서 토크나이징한 토큰들을 모두 리스트에 저장하여 단어장 `V`를 구성한다.
- 이때 `V`에는 중복된 단어가 없어야 한다.  

In [ ]:
# train의 모든 문장 토크나이징해서 단어 사전 만들기
# hint: 파이썬 사전 자료형을 사용하면 간단하게 중복을 제거할 수 있음
# 사용예: {1,2,3}.union({3,4,5}) -> {1,2,3,4,5}

# 파이썬 사전형 변수를 초기화
V = set()

# 모든 학습 데이터에 있는 메일 제목을 순회하면서
# 제목을 토크나이징하고 토큰을 모아서 단어장 V를 완성
for i, d in enumerate(data_train['subject']):
    V = V.union( tokenize(d) )

In [ ]:
# 단어장을 리스트로 변환하고 총 단어장 길이를 출력
V = list(V)
len(V)

4251

In [ ]:
# 리스트로 저장된 단어장 V의 앞 10개 뒤 10개 확인
V[:10], V[-10:]

(['director',
  'e1',
  'possible',
  'mi5',
  'stolen',
  "michael's",
  'executed',
  "'i'm",
  'ultrasound',
  'build'],
 ['doing',
  'coffee',
  'auto',
  'month',
  'benefitting',
  'lord',
  'drop',
  'ben',
  'streams',
  '37771'])

- 단어장 `V`를 이용해서 `V`에서 단어의 위치를 나타내는 인덱스를 입력하면 단어를 돌려주는 `id2token` 사전과 반대로 단어를 입력하면 인덱스를 돌려주는 `token2id`를 생성한다.

In [ ]:
id2token = {i:v for i, v in enumerate(V) }
token2id = {v:i for i, v in enumerate(V)}

In [ ]:
id2token[10], token2id['annual']

('annual', 10)

## 사전확률 $P(C)$ 계산

- 사전 확률은 `data_train`에 있는 스팸 메일 수와 햄 메일 수의 비율로 간단하게 계산할 수 있다.

- `data_train`은 판다스 데이터프레임 변수이므로 데이터프레임 변수를 다루는것에 익숙치 않다면 넘파이 어레이로 변경해서 처리할 수 있다.

- `data_train.values`로 넘파이 어레이로 변경할 수 있다.
```python
data_train.values
array([['[ILUG] Garden Ornaments | mvcmv', True],
       ['Re: mplayer not working for me', False],
       ['Re: exmh bug?', False],
       ...,
       ['New light on a lost world of shattered icons', False],
       ['Re: [meta-forkage]', False],
       ['[use Perl] Headlines for 2002-09-14', False]], dtype=object)
```



In [ ]:
# Prior
# data_train의 스팸메일 수와 햄메일 수를 이용해
# P_ham( P(C='HAM') ), P_ham( P(C='SPAM') )을 계산한다.
# 계산 결과 P_ham = 0.853, P_spam = 0.146 정도 나와야 함
num_ham = (data_train['spam'] == False).astype(int).sum()
num_spam = (data_train['spam'] == True).astype(int).sum()

P_ham = num_ham / (num_ham + num_spam)
P_spam = num_spam / (num_ham + num_spam)

P_ham, P_spam

(0.8536666666666667, 0.14633333333333334)

## 가능도 $P(X \mid C)$ 계산

- 가장 중요한 클래스 조건부 확률, 가능도를 계산한다.
- 클래스 조건부 확률이므로 우선 데이터를 스팸과 햄으로 나눈다.

In [ ]:
data_train_spam = data_train[data_train['spam']==True]
data_train_ham = data_train[data_train['spam']==False]

- $j$번째 토큰에 대한 가능도는 다음처럼 정의된다. 여기서 $j$번째는 단어장 `V`에서 해당 토큰의 위치를 의미한다.

$$
P\left(x_j \mid C\right) = \frac{n_C\left(x_j\right) + k}{N_C + 2k}
$$

- 이렇게 가능도를 계산하는 모델을 베르누이 나이브 베이즈라고 한다.

- 위 식에서 $n_C\left(x_j\right)$는 단어장 기준 $j$번때 토큰이 학습 데이터 셋 문서 중 $C$ 클래스 문서에 나타난 수를 의미하고 $N_C$는 학습 데이터 셋의 $C$ 클래스인 문서 수를 의미한다.

- 예를 들어 스팸메일에 대해서 정리하면 다음과 같다.

$$
P\left(x_j \mid C=Spam\right) = \frac{x_j\text{를 포함하는 스펨메일 수}}{x_j\text{를 포함하는 스팸메일 수} + x_j\text{를 포함하지 않는 스펨메일 수}}
$$

- $k$는 스무딩 팩터로 $n_C\left(x_j\right)$가 0이 되더라도 가능도 값이 0이 됨을 막는 역할을 하게 된다. 다시 말해 해당 클래스 문서에 한번도 나타나지 않는 단어라 하더라도 가능도 값을 0으로는 만들지 않겠다는 것이다. 만약 어떤 토큰의 가능도가 0이 되어 버리면 나이브 가정에 의해 전체 가능도를 각 토큰의 가능도의 곱으로 계산할 때 무조건 문서에 대한 가능도가 0이 되는 문제가 생기게 되는데 이를 방지하기 위함이다.

$$
P\left(x_j \mid C=Spam\right) = \frac{x_j\text{를 포함하는 스펨메일 수}+k}{(x_j\text{를 포함하는 스팸메일 수}+k) + (x_j\text{를 포함하지 않는 스펨메일 수}+k)}
$$

- 다음 순서를 따라 진행한다.
    - 전체 사이즈 `(sample_size, vocab_size)`인 0으로 채워진 행렬을 만든다.
    - 샘플에 대한 인덱스 `i`, 토큰 위치에 대한 인덱스 `j`를 이용해 단어가 등장한 위치에 1을 기록한다.
    - 전체 행렬이 완성되면 행방향으로 모두 더하고 스무딩을 적용하고 반환한다.
    
- 위 구현 과정은 예시일 뿐 다른 방식으로 구현해도 상관없다.

In [ ]:
def get_likelihood(data, vocab_size, k=0.5):
    # data: 클래스로 분루된 샘플들
    # vocab_size: 단어장의 크기
    # k: smoothing factor

    likelihood = np.zeros((data.shape[0], vocab_size))

    # data의 'subject' 항목을 for문으로 순회하면서
    # subject를 token으로 바꾸고
    # token들을 다시 j인덱스로 순회하면
    # i가 문서번호, j가 토큰 번호가 됨
    # 이 상태에서
    # likelihood[i,j] 에 i번째 문서에 j번째 토큰이 나타나면 1씩 더해준다.
    for i, x in enumerate(data['subject']):
        # i: 문서 번호
        # x: 문서
        tokens = tokenize(x) # 중복 단어 제거됨
        for t in tokens:
            j = token2id[t]
            likelihood[i, j] += 1

    # likelihood에 0, 1만 기록되게 되고 axis=0으로 sum하여 각 토큰이
    # 특정 클래스에서 등장한 횟수를 얻을 수 있음
    # 이후 스모딩 팩터 k를 위 수식처럼 적용
    likelihood = (likelihood.sum(axis=0) + k) / (data.shape[0] + 2*k)

    return likelihood

In [ ]:
# 스무딩 팩터를 0.5로 지정하고 모든 토큰에 대한 클래스 조건부 확률을 계산한다.
k = 0.5
P_X_bar_spam = get_likelihood(data_train_spam, len(V), k)
P_X_bar_ham = get_likelihood(data_train_ham, len(V), k)

In [ ]:
# 각 확률이 토큰 개수만큼 계산되었는지 확인한다.
P_X_bar_spam.shape, P_X_bar_ham.shape

((4251,), (4251,))

In [ ]:
# 제대로 계산되었다면 스무딩에 의해 스팸메일에 한번도 등장하지 않는 토큰이
# 약 0.001136 정도의 확률을 가지게 되므로
# 0.00114보다 큰 확률을 가지는 단어를 확인해본다.

# P_X_bar_spam[P_X_bar_spam > 0.00114]

np.array(V)[P_X_bar_spam > 0.00114][:15]

array(['e1', 'possible', 'build', 'earn', 'ccaxc', 'singles', '8119',
       'rekq96sjre5', 'caught', '08', 'iiu', '206', '000', 'try',
       'software'], dtype='<U51')

In [ ]:
# 5배 정도 높은 조건부 확률을 가지는 단어를 조사
# 스팸에 포함될 듯한 단어거 조금은 보이는가?
np.array(V)[P_X_bar_spam > 0.005][:15]

array(['possible', 'earn', 'singles', '000', 'try', 'software', 'friend',
       'breast', 'secret', 'why', 'just', 'sps', 'available', 'safe',
       'risk'], dtype='<U51')

## 나이브베이즈 분류기 작성

- 계산된 `P_X_bar_spam`, `P_X_bar_ham`을 이용해 나이브 베이즈 분류기를 작성한다.

- `naive_bayes_clf(X)`는 문장 `X`를 받아서 다음 순서로 진행한다.

- `X`를 토큰화하여 단어장 V길이의 0, 1로 채워진 특징 벡터로 변환
- 스팸 클래스에 대한 가능도를 다음 식으로 계산한다.

$$
P(X \mid C=Spam) = P(v_1 \mid C=Spam) \times P(v_2 \mid C=Spam) \times \cdots \times P(v_{|V|} \mid C=Spam)
$$

- 위 식에서 $v_1$, ..., $v_{|V|}$는 단어장의 단어를 나타낸다.

- 가능도 항 $P(v_1 \mid C=Spam)$은 단어 $v_1$이 스팸에 등장할 확률을 의미한다. 만약 주어진 문서 $X$에 $v_1$이 등장하지 않았으면 $P(v_1 \mid C=Spam)$를 $1-P(v_1 \mid C=Spam)$로 바꿔서 곱한다.

- 따라서 식을 다음으로 변경한다. 간략한 표기를 위해 $C=spam$을 $S$로 바꿔적었다.

$$
P(X \mid S) = P(v_1 \mid S)^{x_1}(1-P(v_1 \mid S))^{1-x_1} \times P(v_2 \mid S)^{x_2}(1-P(v_2 \mid S))^{1-x_2} \times \cdots \times P(v_{|V|} \mid S)^{x_{|V|}}(1-P(v_{|V|} \mid S))^{1-x_{|V|}}
$$

- 위 식에서 $x_1$, ..., $x_{|V|}$는 $X$가 특징 벡터로 바뀌었을 때 각 자리에 해당하는 0 또는 1을 가지는 특징값들이다.

- 위 식은 확률값이 계속 곱해지는 식이기 때문에 매우 작은 숫자를 곱하게 되 수치적으로 불안정하다. 로그를 취해 덧셈으로 바꾼다.

$$
\log P(X \mid S) = \sum_i x_i \log P(v_i | S) + (1-x_i) \log (1-P(v_i | S))
$$

- 계산을 마친후 양변에 $\exp$를 취한다.

$$
P(X \mid S) = \exp\left(\sum_i x_i \log P(v_i | S) + (1-x_i) \log (1-P(v_i | S)) \right) \tag{1}
$$

- 같은 방식으로 $X$가 정상 메일(Ham)일 가능도도 계산한다.

$$
P(X \mid H) = \exp\left(\sum_i x_i \log P(v_i | H) + (1-x_i) \log (1-P(v_i | H)) \right) \tag{2}
$$

- 최종적으로 다음 사후 확률을 반환 한다.

$$
P(S \mid X) = \frac{P(X \mid S) P(S)}{P(X \mid S) P(S) + P(X \mid H) P(H)} \tag{3}
$$

$$
P(H \mid X) = \frac{P(X \mid H) P(H)}{P(X \mid S) P(S) + P(X \mid H) P(H)} \tag{4}
$$


In [ ]:
def naive_bayes_clf(X):
    # sentence to features
    tokens = tokenize(X)
    token_ids = [token2id[t]  for t in tokens if token2id.get(t, None)]
    feature = np.zeros(len(V))

    # 여기서 문장 X가 [0, 1, 0, 1, ...] 이런식의 특성 벡터로 변하게 됨
    # 벡터의 길이는 V에 들어 있는 토큰 수
    feature[token_ids] = 1.

    ############################################################################
    # calc. P(X|S) and P(X|H) eq(1),(2)
    ############################################################################

    # vector version
    likelihood_spam = np.exp(np.sum(np.log(feature*P_X_bar_spam + (1-feature)*(1-P_X_bar_spam))))
    likelihood_ham = np.exp(np.sum(np.log(feature*P_X_bar_ham + (1-feature)*(1-P_X_bar_ham))))

    # # for loop version
    # likelihood_spam = 0.0
    # for i, xi in enumerate(feature):
    #     if xi == 1:
    #         likelihood_spam += np.log(P_X_bar_spam[i])
    #     else:
    #         likelihood_spam += np.log(1-P_X_bar_spam[i])
    # likelihood_spam = np.exp(likelihood_spam)

    # likelihood_ham = 0.0
    # for i, xi in enumerate(feature):
    #     if xi == 1:
    #         likelihood_ham += np.log(P_X_bar_ham[i])
    #     else:
    #         likelihood_ham += np.log(1-P_X_bar_ham[i])
    # likelihood_ham = np.exp(likelihood_ham)


    ############################################################################
    # calc. P(X|S)P(S) and P(X|H)P(H)
    ############################################################################
    likelihood_spam = likelihood_spam*P_spam
    likelihood_ham = likelihood_ham*P_ham


    ############################################################################
    # normalize `likelihood_spam`, `likelihood_ham` eq(3),(4)
    ############################################################################
    return np.array([likelihood_ham, likelihood_spam]) / (likelihood_ham + likelihood_spam)


## 학습, 테스트 세트에 대해서 성능 평가

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
%%time
y_train = []
preds_train = []

for sample in data_train.values:
    title, target = sample
    y_train.append(target)

    pred = naive_bayes_clf(title)
    if pred[0] < pred[1]:
        preds_train.append(True)
    else:
        preds_train.append(False)


CPU times: user 464 ms, sys: 7.89 ms, total: 472 ms
Wall time: 474 ms


In [ ]:
%%time
y_test = []
preds_test = []

for sample in data_test.values:
    title, target = sample
    y_test.append(target)

    pred = naive_bayes_clf(title)
    if pred[0] < pred[1]:
        preds_test.append(True)
    else:
        preds_test.append(False)


CPU times: user 70.4 ms, sys: 962 µs, total: 71.3 ms
Wall time: 73 ms


In [ ]:
print( classification_report(y_train, preds_train) )
confusion_matrix(y_train, preds_train)

              precision    recall  f1-score   support

       False       0.97      0.99      0.98      2561
        True       0.92      0.83      0.87       439

    accuracy                           0.96      3000
   macro avg       0.95      0.91      0.93      3000
weighted avg       0.96      0.96      0.96      3000



array([[2530,   31],
       [  74,  365]])

In [ ]:
print( classification_report(y_test, preds_test) )
confusion_matrix(y_test, preds_test)

              precision    recall  f1-score   support

       False       0.94      0.98      0.96       359
        True       0.85      0.62      0.72        64

    accuracy                           0.93       423
   macro avg       0.89      0.80      0.84       423
weighted avg       0.92      0.93      0.92       423



array([[352,   7],
       [ 24,  40]])

## 테스트

- 학습세트와 테스트 세트에 없는 임의의 제목에 대해서 테스트

- 다음 20개 샘플은 학습데이터 `spam.csv`에는 없는 완전히 새로운 메일 제목을 ChatGPT로 생성한 것임

- 샘플은 `(제목, 스팸유무)` 로 구성됨

- 학습된 모델을 사용하여 각 샘플에 대해 다음 형식으로 출력
```
정답    예측     HAM확률 SPAM확률  제목
[SPAM], [SPAM]   0.03%,  99.97%:   Double your income instantly - Click here to find out how!
```

In [ ]:
test_samples = [
    ("Congratulations! You've won $1,000,000!",'SPAM'),
    ("Unlock your secret fortune NOW!",'SPAM'),
    ("Hot singles in your area are waiting for you!",'SPAM'),
    ("You won't believe what celebrities are hiding!",'SPAM'),
    ("Lose 30 pounds in just 3 days with this miracle pill!",'SPAM'),
    ("Exclusive VIP offer: Get rich overnight!",'SPAM'),
    ("Warning! Your computer is infected with 3 viruses!",'SPAM'),
    ("Claim your free iPhone 13 now - limited time offer!",'SPAM'),
    ("You have unread messages from the IRS - Act now!",'SPAM'),
    ("Double your income instantly - Click here to find out how!",'SPAM'),
    ("Meeting Agenda for Monday, September 18th", 'HAM'),
    ("Invoice #12345 for Services Rendered", 'HAM'),
    ("Weekly Project Update: September 1-7", 'HAM'),
    ("Feedback Requested: Q3 Marketing Strategy", 'HAM'),
    ("Request for Order Delivery Confirmation - Order Number #JPXU-2596", 'HAM'),
    ("RSVP: End-of-Year Party", 'HAM'),
    ("New Course Offering: Introduction to Python Programming", 'HAM'),
    ("Your Flight Itinerary: Confirmation #ABC123", 'HAM'),
    ("Job Application: Smith for AI Director", 'HAM'),
    ("Newsletter: [Company Name] September Updates", 'HAM')]

random.shuffle(test_samples)

correct = 0
for sample in test_samples:
    pred = naive_bayes_clf(sample[0])

    if pred[0] < pred[1]:
        if sample[1] != 'SPAM':
            color_code_s = "\033[31m"
            color_code_e = "\033[0m"
        else:
            color_code_s = "\033[32m"
            color_code_e = "\033[0m"

        # 앞에 정답 뒤에 예측
        print(f"{color_code_s}[{sample[1]:>4}], [SPAM] {pred[0]*100:6.2f}%, {pred[1]*100:6.2f}%: {sample[0]}{color_code_e}")
    else:
        if sample[1] != 'HAM':
            color_code_s = "\033[31m"
            color_code_e = "\033[0m"
        else:
            color_code_s = "\033[32m"
            color_code_e = "\033[0m"

        print(f"{color_code_s}[{sample[1]:>4}], [ HAM] {pred[0]*100:6.2f}%, {pred[1]*100:6.2f}%: {sample[0]}{color_code_e}")


[ HAM], [ HAM]  87.44%,  12.56%: Request for Order Delivery Confirmation - Order Number #JPXU-2596
[SPAM], [SPAM]   8.22%,  91.78%: You have unread messages from the IRS - Act now!
[ HAM], [ HAM]  98.44%,   1.56%: Job Application: Smith for AI Director
[ HAM], [ HAM]  99.86%,   0.14%: New Course Offering: Introduction to Python Programming
[SPAM], [SPAM]   0.41%,  99.59%: Exclusive VIP offer: Get rich overnight!
[SPAM], [SPAM]   0.38%,  99.62%: Hot singles in your area are waiting for you!
[SPAM], [SPAM]  35.40%,  64.60%: You won't believe what celebrities are hiding!
[SPAM], [SPAM]   0.15%,  99.85%: Lose 30 pounds in just 3 days with this miracle pill!
[ HAM], [ HAM]  99.79%,   0.21%: Newsletter: [Company Name] September Updates
[ HAM], [ HAM]  99.97%,   0.03%: Weekly Project Update: September 1-7
[SPAM], [SPAM]   0.91%,  99.09%: Unlock your secret fortune NOW!
[SPAM], [SPAM]   0.03%,  99.97%: Double your income instantly - Click here to find out how!
[ HAM], [ HAM]  99.75%,   0.25%: 

## scikit-learn 으로 검증

- 지금까지 코딩한 모델의 유효성을 검증하기 위해 scikit-learn에서 제공하는 나이브베이즈인 `BernoulliNB`를 사용하여 결과를 비교한다.

- 먼저 문장을 길이 (|V|,)인 특징 벡터로 바꾸는 함수를 작성한다.

In [ ]:
def transform_to_features(data):
    X = []
    y = []

    for d in data.values:
        title, target = d

        tokens = tokenize(title)
        token_ids = [token2id[t]  for t in tokens if token2id.get(t, None)]
        feature = np.zeros(len(V))
        X.append(feature)

        y.append(0 if target == False else 1)

    return np.array(X), np.array(y)

- 작성된 함수에 `data_train`, `data_test`를 입력하고 문장들을 특징벡터로 변환한다.

In [ ]:
X_train, y_train =  transform_to_features(data_train)
X_test, y_test =  transform_to_features(data_test)

X_train.shape, X_test.shape

((3000, 4251), (423, 4251))

- scikit-learn으로 부터 모델을 로딩하고 준비된 학습 데이터를 사용하여 `fit`시키고 결과를 확인한다.

- 얻어진 결과와 직접 만든 모델의 결과를 비교해본다.

In [ ]:
from sklearn.naive_bayes import BernoulliNB

In [ ]:
clf = BernoulliNB(alpha=0.5)

In [ ]:
clf.fit(X_train, y_train)

BernoulliNB(alpha=0.5)

In [ ]:
pred_train = clf.predict(X_train)
pred_test = clf.predict(X_test)

In [ ]:
print( classification_report(y_train, preds_train) )
confusion_matrix(y_train, preds_train)

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      2561
           1       0.92      0.83      0.87       439

    accuracy                           0.96      3000
   macro avg       0.95      0.91      0.93      3000
weighted avg       0.96      0.96      0.96      3000



array([[2530,   31],
       [  74,  365]])

In [ ]:
print( classification_report(y_test, preds_test) )
confusion_matrix(y_test, preds_test)

              precision    recall  f1-score   support

           0       0.94      0.98      0.96       359
           1       0.85      0.62      0.72        64

    accuracy                           0.93       423
   macro avg       0.89      0.80      0.84       423
weighted avg       0.92      0.93      0.92       423



array([[352,   7],
       [ 24,  40]])